<center><font size = "10"> Week 11 - Networks <center>
<center><font size = "8">Tutorial 01: Spontaneous activity <center>
    
    

When the brain is not actively processing sensory input or performing some task it is nonetheless constantly active. In fact, most of the energy consumed by your brain is due to this spontaneous activity. What the point of all this spontaneous activity is is not yet clear and we have a lot to learn about it.

In brain modeling the overall depolarization and variability of activity coming into a brain region of interest from distant regions is going to strongly influence the network dynamics and firing behavior. However, explicitly modeling all input regions just to get the dynamics of a small region correct is computationally expensive. It is therefore common practice to model this background of distant input as "noise". In reality, it is lkely that some important information is carried by these inputs, but for the purposes of the model we are agnostic to it.

Usually, coming up with a noise model and ensuring it reproduces the spontaneous activity of the brain region is done before you apply a stimulus.

In [ ]:
from neuron import h
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(1)

In [ ]:
class Cell:
    def __init__(self, filename, name, cell_type):
        self.name = name
        self.type = cell_type
        self.synapses = []
        self.build_morphology(filename)
        self.biophysics()

    def build_morphology(self, filename):
        h.load_file("stdlib.hoc")
        h.load_file("import3d.hoc")
        if filename.endswith('.asc'):
            morph_reader = h.Import3d_Neurolucida3()
        else:
            morph_reader = h.Import3d_SWC_read()
        morph_reader.input(filename)
        i3d = h.Import3d_GUI(morph_reader, 0)
        i3d.instantiate(
            self
        )  # Notice this change to be able to instantiate several cells
        # all_sections should have the same order as neurom
        self.all_sections = self.soma + self.axon + self.dend
        if hasattr(self, 'apic'):
            self.all_sections += self.apic
        print(len(self.dend), len(self.all_sections))
        self.efferent_synapses = []
        self.afferent_synapses = []

    def biophysics(self):
        for sec in h.allsec():
            sec.Ra = 100  # Axial resistance in Ohm * cm
            sec.cm = 1  # Membrane capacitance in micro Farads / cm^2
            sec.insert("pas")
            for seg in sec:
                seg.pas.g = 0.00003
                seg.pas.e = -75

        # Insert passive current in the dendrite
        for sec in self.soma:
            sec.insert("hh")
            for seg in sec:
                seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

        if hasattr(self, "apic"):
            for sec in self.apic:
                sec.insert("hh")
                for seg in sec:
                    seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                    seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

        for sec in self.dend:
            sec.insert("hh")
            for seg in sec:
                seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

        for sec in self.axon:
            sec.insert("hh")
            for seg in sec:
                seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

In [ ]:
Pyr1 = Cell("Pyr_01.swc", "Pyr1", "Pyr")
Pyr2 = Cell("Pyr_02.swc", "Pyr2", "Pyr")
Int1 = Cell("Int_01.swc", "Int1", "Int")
cells_Pyr = [Pyr1, Pyr2]
cells_Int = [Int1]
cells = cells_Pyr + cells_Int

## Creating background "noise"

In this case we are creating background noise that fluctuates rapidly. If the noise current at each timestep were independent of the noise at preceding/succeeding times, the noise would quite quickly smooth out to a constant when the size of the time step is small. Actual neural activity is also not temporally independent, but if a neuron was firing a few milliseconds ago, the odds are higher that it will be firing again. So, for this tutorial we instead have an input current that changes a little bit at random with each timestep.

In the cortex, long-range inputs are either excitatory or modulatory. We are working with cortical cells at the moment, so let us assume that our noise current is non-negative.

__Further reading:__

More sophisticated and data-driven noise models have been used in https://www.biorxiv.org/content/10.1101/2023.05.17.541168v3.abstract and https://www.science.org/doi/full/10.1126/sciadv.abq7592 . 

In [ ]:
bkg_stims = []
bkg_currents = []
for cell in cells:
    stim = h.IClamp(cell.soma[0](0.5))
    stim.delay = 0
    stim.dur = 1e9
    last = 0
    stimulus_by_time = []
    decay = 0.5
    amp = 0.9
    for st in np.arange(0, 3000, h.dt):
        change = np.random.normal() * amp * h.dt
        new = last - (last * decay * h.dt) + change
        if new < 0:
            new = 0
        last = new
        stimulus_by_time.append(new)
    stim_current = h.Vector(stimulus_by_time)
    bkg_currents.append(stim_current)
    plt.plot(stimulus_by_time, alpha=0.1)
    plt.xlim(0, 1000)
    stim_current.play(stim._ref_amp, h.dt)
    bkg_stims.append(stim)

print(bkg_stims)

In [ ]:
def run_simulation(simulation_time):
    Vm = []
    for cell in cells:
        v = h.Vector()
        v.record(cell.soma[0](0.5)._ref_v)
        Vm.append(v)


    time = h.Vector()
    time.record(h._ref_t)


    h.finitialize(-65)
    h.load_file('stdrun.hoc')
    h.continuerun(simulation_time)

    return Vm, time



Vm, time = run_simulation(200)

In [ ]:
synapses = pd.DataFrame(columns=['@source_node', '@target_node'])

def connectivity_plot(axis, synapses):
    for (pre, post), conn in synapses.reset_index().groupby(['@source_node', '@target_node'])['index'].count().items():
        axis.plot([0, 1], [-pre, -post], linewidth=conn)
    axis.set_yticks([])
    axis.set_xticks([])


def plot_results(Vm, time):

    n_subplots = len(Vm)
    f = plt.figure(figsize=(24, 12))
    axis=None
    axes = []
    for i, v in enumerate(Vm):
        if axis is None:
            axis = plt.subplot(n_subplots, 2, 2*i + 2)
        else:
            axis = plt.subplot(n_subplots, 2, 2*i + 2, sharex=axis)
        axis.plot(time, v)
        axis.set_ylabel('$V_m (mV)$')
        axes.append(axis)
    axis.set_xlabel('time (ms)')
    ax4 = plt.subplot(1, 2, 1)
    connectivity_plot(ax4, synapses)
    
    ax4.set_ylabel('Neuron')
    ax4.set_title('connections')
    axes.append(ax4)
    return f, axes
 
plot_results(Vm, time)

Note in the above graph that the same noise current has a much stronger effect in different cells! (three plots at the right). This is why most real noise models are often scaled based on the rheobase of the cell model.

In [ ]:
connprobs = np.array([[0.8, #pyr-pyr 
                       0.9], # pyr-int,
                      [0.8, # int-pyr
                       0]]) # int-int

syn_per_conn = np.array([[2, 3], [4, 2]])


synapses = []
for pregid, pre in enumerate(cells):
    for postgid, post in enumerate(cells):
        if pre == post:
            continue
        if pre in cells_Pyr:
            i = 0
        else:
            i = 1
        if post in cells_Pyr:
            j = 0
        else:
            j = 1
        connprob = connprobs[i, j]
        if np.random.uniform(0, 1) > connprob:
            continue
        if hasattr(post, 'apic'):
            post_cell_dends = post.dend + post.apic
        else:
            post_cell_dends = post.dend
        post_cell_offset = 1 + len(post.axon)
        print(len(post.all_sections), len(post.dend), len(post_cell_dends), post_cell_offset, len(post_cell_dends) + post_cell_offset)
        for _ in range(syn_per_conn[i, j]):
            # done this way because it's similar to the data you will work with
            synapses.append({
                '@source_node': int(pregid),
                '@target_node': int(postgid),
                'efferent_section_id': np.random.randint(1, 1 + len(pre.axon)),
                'afferent_section_id': np.random.randint(post_cell_offset, post_cell_offset + len(post_cell_dends)),
                'afferent_section_pos': np.random.uniform(0, 1), 
                'efferent_section_pos': np.random.uniform(0, 1)})

            
synapses = pd.DataFrame(synapses)
synapses

In [ ]:
def place_synapse(pre_cell, post_cell, synapse_data):

    post_segment = post_cell.all_sections[int(synapse['afferent_section_id'])](synapse['afferent_section_pos'])
    pre_section = pre_cell.all_sections[int(synapse['efferent_section_id'])]
    pre_segment = pre_section(synapse['efferent_section_pos'])
    syn = h.ExpSyn(post_segment)
    nc = h.NetCon(pre_segment._ref_v, syn, sec=pre_section)
    nc.weight[0] = 0.5
    nc.delay=5 # a 5ms delay gives us more interpretable results, but be aware synaptic transmission is usually a lot faster
    pre_cell.efferent_synapses.append((syn, nc))
    post_cell.afferent_synapses.append((syn, nc))

    return syn, nc

synapse_models = []
netcons = []
for _, synapse in synapses.iterrows():
    source = cells[int(synapse['@source_node'])]
    target = cells[int(synapse['@target_node'])]
    syn, nc = place_synapse(source, target, synapse)
    synapse_models.append(syn)
    netcons.append(nc)
    

In [ ]:
Vm, time = run_simulation(200)
plot_results(Vm, time)

# inhibition

Most interneurons are inhibitory, with the reversal potential of the GABA-A receptor at around -70 mV in adult animals.

In this case we see a very modest inhibitory effect, as the voltage is mostly already close to -70mV

In [ ]:
for eff, nc in Int1.efferent_synapses:
    eff.e = -70


In [ ]:
Vm, time = run_simulation(200)

f, a = plot_results(Vm, time)
